In [27]:
import pandas as pd

YEARS = [2019]

data = pd.DataFrame()

#ingest
for year in YEARS:
    data = data.append(pd.read_csv(f"data/play_by_play_cleaned_{year}.csv.gz", compression='gzip', low_memory=False).reset_index(drop=True), sort=True)

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 300)
data.reset_index(drop=True, inplace=True)

In [15]:
# filter data for game script
DOWN_UPPER = 3
NEUTRAL = False
if NEUTRAL:
    neutral = data.loc[ (data.play_type!="no_play") & (data.qtr<5) &
                           (data.down<=DOWN_UPPER) & (data.half_seconds_remaining>120) &
                                     (data.wp>=.2) & (data.wp<=.8)].reset_index(drop=True)
else:
     neutral = data.loc[ (data.play_type!="no_play") & (data.qtr<5)].reset_index(drop=True)

In [16]:
import numpy as np
def epa_success(epa):
    return 1 if epa>0 else 0
sea = neutral.loc[(neutral.posteam=="SEA")]
qtrData = sea.groupby(['posteam', 'down']).agg(
                play_count = ('play_id', 'count'),
                pass_rate = ('pass', 'mean'),
                total_epa = ('epa', 'sum'),
                avg_epa = ('epa', 'mean'),
                epa_succ_rate = ('epa', lambda x: (x.map(epa_success).mean()))
            ).round(2).reset_index()
qtrData

,posteam,down,play_count,pass_rate,total_epa,avg_epa,epa_succ_rate
0,SEA,1.0,307,0.45,4.76,0.02,0.43
1,SEA,2.0,227,0.57,31.51,0.14,0.51
2,SEA,3.0,132,0.80,-14.20,-0.11,0.44
